In [ ]:
#! pip install torch rawpy opencv-python matplotlib skicit-image scipy 

In [7]:
import os
import sys

sys.path.insert(0, "../Uformer-RSBlur")

# ISP 유틸들 로드
from ISP.utils import *
import rawpy
import cv2
import torch
import matplotlib.pyplot as plt

# 시각화 코드
def viz_two_images(img1, img2, title1, title2):
    fig = plt.figure(figsize=(16, 16))
    rows = 1; cols = 2

    ax1 = fig.add_subplot(rows, cols, 1)
    ax1.set_title(title1)
    
    if len(img1.shape) == 2:
        image1 = ax1.imshow(img1, 'gray')
    else:
        image1 = ax1.imshow(img1)

    ax2 = fig.add_subplot(rows, cols, 2)
    ax2.set_title(title2)
    
    if len(img2.shape) == 2:
        image2 = ax2.imshow(img2, 'gray')
    else:
        image2 = ax2.imshow(img2)
    fig.tight_layout()
    plt.show()
demosaic = Demosaic()

# Read raw image and sRGB image

본 튜토리얼의 목표는 AR73의 RAW 이미지를 처리하여 카메라에서 얻은 JPEG 이미지와 최대한 비슷한 출력을 얻는 ISP를 구성하는 것입니다.

RealBlur 데이터셋에서 사용 된 Sony A7R3 카메라를 활용해 샘플 이미지를 미리 촬영해 놓았습니다. 보통의 카메라에는 RAW + JPEG 이미지를 동시에 촬영하는 기능이 있으며, 이를 통해 JPEG 이미지와 그에 해당하는 RAW 이미지를 동시에 촬영하여 활용합니다.

rawpy는 raw 이미지를 처리하는데 많이 활용되는 libraw (https://www.libraw.org/about) 의 python wrapper 입니다.


In [8]:
raw_path = 'notebooks/samples/CLN00001.ARW' # 샘플 raw image
raw = rawpy.imread(raw_path)
a7r3_raw = raw.raw_image_visible.copy()
a7r3_raw = a7r3_raw[8:-8, 8:-8]

srgb_path = 'notebooks/samples/CLN00001.JPG' # 샘플 srgb image
a7r3_srgb = cv2.imread(srgb_path, cv2.IMREAD_COLOR).astype('float32')/255
a7r3_srgb = cv2.cvtColor(a7r3_srgb, cv2.COLOR_BGR2RGB)

LibRawIOError: b'Input/output error'

In [ ]:
a7r3_srgb.shape,a7r3_raw.shape

두 장의 이미지를 시각화 하면 밑과 같습니다.

In [ ]:
viz_two_images(a7r3_srgb, a7r3_raw, 'srgb image', 'raw image')

# Bayer pattern

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/1/1c/Bayer_pattern_on_sensor_profile.svg/525px-Bayer_pattern_on_sensor_profile.svg.png'>

이미징 센서는 빛의 양을 측정하는 장치이며 RGB 스펙트럼에 해당하는 빛의 양을 측정 하기 위해 각 센서의 앞에 작은 RGB 색에 맞는 필터를 부착합니다. 이러한 필터의 구성 중 4픽셀을 묶음으로 RGGB 와 같이 R 픽셀, G 픽셀 2개, B 픽셀 하나로 구성되어져 있는 패턴을 Bayer pattern이라고 합니다. 

이러한 Bayer pattern의 이미지를 이후 디모자이크 과정을 통해 비어있는 값을 채움으로써 RGB이미지로 변환하게 됩니다.

A7R3 카메라의 이미지는 RGGB에 해당하는 Bayer pattern으로 저장되어 있습니다. 크롭하여 자세히 살펴보면 격자 무늬 처럼 보이는 Bayer 패턴을 볼 수 있습니다.

In [ ]:
viz_two_images(a7r3_srgb[1570:1670, 2070:2170,:], a7r3_raw[1570:1670, 2070:2170], 'srgb image', 'raw image')

이를 ```python from ISP.utils import *``` 에 구현되어져 있는 torch 함수 들로 처리하기 위해 torch 텐서로 변환 합니다. Raw 이미지들을 처리하기 편하도록 RGGB 패턴에 따라 4채널로 스택합니다.

In [ ]:
# numpy to torch tensor
a7r3_raw_pt = torch.from_numpy(a7r3_raw.copy().astype('float32'))

red = a7r3_raw_pt[0::2, 0::2] # RGGB 패턴에서 R에 해당
green_red = a7r3_raw_pt[0::2, 1::2] # RGGB 패턴에서 G1에 해당
green_blue = a7r3_raw_pt[1::2, 0::2] # RGGB 패턴에서 G2에 해당
blue = a7r3_raw_pt[1::2, 1::2] # RGGB 패턴에서 B에 해당

img_raw = torch.stack([red, green_red, green_blue, blue], dim=2)

# Black and white level normalization

In [ ]:
a7r3_raw.min(), a7r3_raw.max()


Raw 이미지들은 센서의 스펙에 따라 12 bit, 14 bit, 16 bit 등으로 저장되어져 있습니다. a7r3의 경우 최대값이 16383 인 것으로 보아 14 비트로 저장되어져 있는 것을 알 수 있습니다.

하지만 14비트로 저장되어져 있더라도, 실제 센서의 최소, 최대 값은 0, 16383이 아닙니다. 센서의 최소, 최대 값을 각각 black, white level 이라고 합니다. 이러한 값들은 보통 raw 이미지의 메타데이터에 저장되어져 있으며 센서에 맞도록 black, white level으로 normalization을 수행 해 주어야 합니다.

상업용 카메라의 경우에는 RAW 파일에 다양한 메타데이터들이 저장되어져 있습니다.
exiftool 은 이러한 메타데이터를 읽는 툴이며 raw 이미지에 대한 다양한 정보가 저장되어져 있습니다.

In [ ]:
#! apt-get install exiftool 
!exiftool notebooks/samples/CLN00001.ARW


```bash
Black Level                     : 512 512 512 512
White Level                     : 15360 15360 15360
```

이를 통해 raw 영상을 0 ~ 1 값으로 normalization을 수행합니다. 밑의 과정은 단순 min-max normalization과 동일합니다.

In [ ]:
black_level = 512
white_level = 15360

img_normalize = (img_raw - black_level) / (white_level - black_level)

In [ ]:
img_normalize.min(), img_normalize.max()

칼라 영상으로 이미지를 보기 위해 demosaic을 수행하고 시각화 하였습니다.

In [ ]:
# for visualziation
img_normalize_demosaic = torch.nn.functional.pixel_shuffle(img_normalize.permute(2, 0, 1).unsqueeze(0), 2)
img_normalize_demosaic = demosaic.forward(img_normalize_demosaic, pattern='RGGB').squeeze(0).permute(1, 2, 0)
viz_two_images(a7r3_srgb, lin2rgb_pt(img_normalize_demosaic), 'srgb image', 'normalized image')

화이트 밸런싱을 수행하지 않아 흰색 배경들이 녹색처럼 보이는 결과를 얻을 수 있습니다. 

# White balance

사람은 조명이 달라지더라도 같은 색으로 지각할 수 있는 색 항상성을 가지고 있습니다. 하지만 카메라는 이러한 일을 할 수 없어 화이트 밸런싱을 수행 해 주어야 합니다. 좀 더 쉬운 말로 한다면, 현재 흰색이 어떤 색인지를 알려주는 것과 같습니다.

다양한 화이트밸런싱을 위한 알고리즘이 있으며 이 예제에서는 JPG와 최대한 맞추기 위해 카메라에서 자동으로 계산해준 값을 사용합니다.

역시 화이트밸런싱을 위한 값도 메타데이터에 저장되어져 있습니다.


```bash
WB RGGB Levels                  : 2576 1024 1024 1652
CFA Pattern                     : [Red,Green][Green,Blue]
```

In [ ]:
wb_r = 2576/1024.0 # R 에 대한 값
wb_g = 1.0
wb_b = 1652/1024.0 # B 에 대한 값

# white balance
# RGGB 에 해당하는 패턴에 따라, wb_r, wb_g, wb_b 를 곱해주는 과정
img_wb = img_normalize.clone()
img_wb[:,:,0] = img_wb[:,:,0] * wb_r
img_wb[:,:,1] = img_wb[:,:,1] * wb_g
img_wb[:,:,2] = img_wb[:,:,2] * wb_g
img_wb[:,:,3] = img_wb[:,:,3] * wb_b

bayer_pattern = 'RGGB'
#img_wb = WB_img(img_normalize, bayer_pattern, wb_r, wb_b)


In [ ]:
# for visualziation

img_wb_demosaic = torch.nn.functional.pixel_shuffle(img_wb.permute(2, 0, 1).unsqueeze(0), 2)
img_wb_demosaic = demosaic.forward(img_wb_demosaic, pattern='RGGB').squeeze(0).permute(1, 2, 0)
viz_two_images(a7r3_srgb, lin2rgb_pt(img_wb_demosaic), 'srgb image', 'normalized image')

화이트 밸런싱을 수행하니 이제야 흰색이 제대로 흰색처럼 보이는 것을 알 수 있습니다.

# Demosaic

이전에 설명한 것 처럼 Bayer pattern의 이미지를 이후 디모자이크 과정을 통해 비어있는 값을 채움으로써 RGB이미지로 변환하게 됩니다.

<img src='https://i0.wp.com/theailearner.com/wp-content/uploads/2018/10/demosaicing_procedure.png?resize=768%2C485&ssl=1'>


In [ ]:
img_demosaic = torch.nn.functional.pixel_shuffle(img_wb.permute(2, 0, 1).unsqueeze(0), 2)
img_demosaic = demosaic.forward(img_demosaic, pattern=bayer_pattern).squeeze(0).permute(1, 2, 0)

print(img_wb.shape, img_demosaic.shape)

디모자이킹 이후 모든 픽셀마다 RGB 값들이 채워져 3채널의 이미지인 것을 확인 가능합니다.

In [ ]:
viz_two_images(a7r3_srgb, lin2rgb_pt(img_demosaic), 'srgb image', 'demosaiced image')

드디어 뭔가 정상적인 이미지처럼 보이지만, 각각의 RGB 값들은 실제 지각적 공간에서 정의 된 RGB의 값이 아닙니다.

# Color conversion (rawRGB2linearRGB)

실제 카메라들의 RGB 필터들은 밑과 같이 각각 다른 스펙트럼을 가지고 있습니다.

<img src='https://3.img-dpreview.com/files/p/TS560x560~forums/61592391/4d350ed34a054ef58741965c827b351f'>

예전에, 컴퓨터 비전 수업 시간에 등색 실험에 대해 접한 적이 있으실 겁니다. 이는 사람이 특정 색으로 지각하기 위해 RGB 색을 얼마나 섞어야 하는지에 대한 지각적 실험입니다. 이를 통해 지각적 RGB 색공간을 정의 했었습니다. 

![image.jpg1](https://user-images.githubusercontent.com/60923302/113116008-fd9a1900-9247-11eb-8fc1-5397be878197.png) |![image.jpg2](https://user-images.githubusercontent.com/60923302/113119423-62a33e00-924b-11eb-803c-990594b91954.png)
--- | --- | 

실제 카메라들의 Raw-RGB 스펙트럼은 위의 지각적 RGB 과 상이합니다. 이를 Raw-RGB 색공간을 지각적 RGB 공간으로 맞춰주는 과정을 Color Conversion 이라고 합니다. 보통은 Color Conversion Matrix 라고 불리는 3x3 매트릭스를 통해 Raw-RGB를 XYZ 표준 색 공간으로 변환 합니다.  

밑의 코드는 제가 미리 구해놓은 Color Conversion Matrix로 색공간을 변환한 결과입니다. 이전 결과에 비해 색상이 비비드 해지는 것을 볼 수 있습니다.

In [ ]:
img_demosaic = img_demosaic.clamp(0, 1)
img_IXYZ = img_demosaic @ cam2xyz_realblur_right

# from XYZ to linear RGB
img_IL = img_IXYZ @ M_xyz2lin
img_IL = torch.clamp(img_IL, 0, 1)

viz_two_images(a7r3_srgb, lin2rgb_pt(img_IL), 'srgb image', 'ccm image')

### How to get color conversion matrix (ccm)?

1. Find ccm matrix from libraw library. (https://github.com/LibRaw/LibRaw/blob/2a9a4de21ea7f5d15314da8ee5f27feebf239655/src/tables/colordata.cpp)
2. Estimate ccm by yourself. (refer to Color conversion matrices in digital cameras: a tutorial.)

ILCE-7RM3, ILCE-7RM3A { 6640,-1847,-503,-5238,13010,2474,-993,1673,6527 }

### Notes
* ccm matrix from libraw is supposed to be normalized.

대부분의 상업용 카메라들은 libraw 라는 라이브러리에서 미리 계산 해놓은 매트릭스를 제공합니다. 

A7R3 카메라의 경우는 밑과 같은 값들을 찾을 수 있습니다.

ILCE-7RM3, ILCE-7RM3A { 6640,-1847,-503,-5238,13010,2474,-993,1673,6527 }

만약 libraw에서 찾을 수 없다면 직접 추정 해시는 것을 추천합니다. 위의 사진과 같이 color chart를 촬영하여 추정 할 수 있습니다. 

In [ ]:
def normalize_ccm(libraw_xyz2cam):
    # refer to section 6.1 of Color conversion matrices in digital cameras: a tutorial. Rowland, Optical engineering 2020.
    
    # normalize ccm using white point of D65
    wp_d65_xyz = torch.tensor([0.9504, 1, 1.0888]);
    rgb_wp_d65 = apply_cmatrix(wp_d65_xyz[None,None,:], libraw_xyz2cam)

    c = rgb_wp_d65.max();

    libraw_xyz2cam = libraw_xyz2cam / c;
    
    # Convert ccm of D65 to ccm of auto white balanced images
    wp_rgb = torch.tensor([1.0, 1.0, 1.0]);
    wp_rgb_xyz = lin2xyz(wp_rgb[None,None,:])

    rgb = apply_cmatrix(wp_rgb_xyz[None,None,:], libraw_xyz2cam)
    D_inv = torch.diag(rgb.reshape(3))

    cam2xyz = torch.inverse(libraw_xyz2cam) @ D_inv
        
    return cam2xyz

libraw_xyz2cam = torch.tensor([6640, -1847, -503, -5238, 13010, 2474, -993, 1673, 6527]).reshape(3,3).float() / 10000
cam2xyz = normalize_ccm(libraw_xyz2cam)
cam2xyz_right = cam2xyz.permute(1, 0)

In [ ]:
img_demosaic = img_demosaic.clamp(0, 1)
# img x M_cam2xyz
img_IXYZ = img_demosaic @ cam2xyz_right

# from XYZ to linear RGB
# img x M_xyz2lin
img_IL = img_IXYZ @ M_xyz2lin
img_IL = torch.clamp(img_IL, 0, 1)

viz_two_images(a7r3_srgb, lin2rgb_pt(img_IL), 'srgb image', 'ccm image')

하지만 여전히 카메라에서 얻은 JPG 이미지와는 많은 차이가 있는 것을 볼 수 있습니다. 

# Camera Response Function (Camera non-linearity)

카메라 내부에서는 이미지의 동적 범위를 조절하여 사람이 보기 좋은 이미지를 만들기 위해 gamma correction과 같은 non-linearity가 포함되어 있습니다. 최근의 카메라에서는 보다 나은 사용자 경험을 위해 gamma correction과 매우 다른 함수들을 활용 합니다.

<img src='https://miro.medium.com/v2/resize:fit:786/format:webp/1*Gws3rgBJcUqU_gi4pE6oiw.png'>


이러한 카메라 내부의 Non-linearity를 제대로 추정해서 적용해야 Realistic한 blurred 데이터셋을 만들 수 있습니다.

밑의 예제는, 미리 추정해 놓은 CRF를 적용한 예제입니다.

In [ ]:
img_Irgb = lin2rgb_a7r3_polynomial(img_IL)
img_Irgb = torch.clamp(img_Irgb, 0, 1) # (h, w, c)

viz_two_images(a7r3_srgb, img_Irgb, 'srgb image', 'final image')

# How to estimate CRF

1. Capture color chart images.
2. Conduct polynomial regression between linear-RGB of raw image vs JPEG image of camera.

Color chart를 촬영한 이미지를 통해 CRF를 추정 할 수 있습니다. 본 논문에서는 카메라 내부 에서의 linear-RGB를 sRGB로 변환 하는 과정에만 관심이 있습니다. 이는 실제 카메라에서 얻어진 JPG 이미지와, 이전까지 구해놓은 linear-RGB 이미지 사이의 관계를 통해 추정 가능합니다 

### Note
* Radiometric calibration 등에서 사용되는 camera response function 보다 좁은 의미인 카메라 내부의 Non-linearity 만을 추정합니다. 

# Detect color checker

In [ ]:
#!pip install opencv-python==4.5.1.48
#!pip install colour_checker_detection

In [ ]:
import colour
from colour_checker_detection import detect_colour_checkers_segmentation

image = a7r3_srgb
colour_checker_swatches_data_srgb = detect_colour_checkers_segmentation(image, additional_data=True)

In [ ]:
# color checker 디텍션 결과 시각화
for colour_checker_swatches_data in colour_checker_swatches_data_srgb:
    swatch_colours, colour_checker_image, swatch_masks = (
        colour_checker_swatches_data)

    # Using the additional data to plot the colour checker and masks.
    masks_i = np.zeros(colour_checker_image.shape)
    for i, mask in enumerate(swatch_masks):
        masks_i[mask[0]:mask[1], mask[2]:mask[3], ...] = 1
    colour.plotting.plot_image(
            np.clip(colour_checker_image + masks_i * 0.25, 0, 1));

In [ ]:
colour_checker_swatches_data_srgb = detect_colour_checkers_segmentation(a7r3_srgb, additional_data=True)
colour_checker_swatches_data_linrgb = detect_colour_checkers_segmentation(img_IL.numpy(), additional_data=True)

# mask 만큼의 평균 픽셀 값을 반환 합니다. (4x6 patches)
y = colour_checker_swatches_data_srgb[0].swatch_colours
x = colour_checker_swatches_data_linrgb[0].swatch_colours
x, y

In [ ]:
plt.scatter(x[:,0],y[:,0])
plt.scatter(x[:,1],y[:,1])
plt.scatter(x[:,2],y[:,2])

# Polynomial regression

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression


y = colour_checker_swatches_data_srgb[0].swatch_colours.flatten()
x = colour_checker_swatches_data_linrgb[0].swatch_colours.flatten()

# degree : 거듭제곱의 차수, include_bias : 편향값(1) 추가 여부
poly = PolynomialFeatures(degree=8, include_bias=False)

# 훈련 데이터셋 X_train 의 거듭제곱을 생성한 뒤, 훈련 데이터셋 X_train 에 새로운 변수로 추가
X_train_poly = poly.fit_transform(x[:, None])
lin_reg = LinearRegression(fit_intercept=False)
lin_reg.fit(X_train_poly, y)

x_plot = np.linspace(0, 1, 200)
x_poly = poly.fit_transform(x_plot[:,None])
y_pred = lin_reg.predict(x_poly)
plt.plot(x_plot, y_pred)

#pred_our = lin2rgb_a7r3_polynomial(torch.tensor(x_plot))
#plt.plot(x_plot, pred_our)

plt.scatter(x.flatten(),y.flatten(),color='r',s=6)

# Using two color chart images

exposure을 다르게 하여 여러장의 이미지를 촬영하여 보다 정확한 CRF를 추정할 수 있습니다. 논문에서는 총 11장의 이미지를 활용합니다.


In [ ]:
def raw2Lin(image, wb_r, wb_b):
    
    image = torch.from_numpy(image.copy().astype('float32'))
    
    red = image[0::2, 0::2]
    green_red = image[0::2, 1::2]
    green_blue = image[1::2, 0::2]
    blue = image[1::2, 1::2]

    img_raw = torch.stack([red, green_red, green_blue, blue], dim=2)
    
    # normalize
    black_level = 512
    white_level = 15360

    img_normalize = (img_raw - black_level) / (white_level - black_level)
    
    # White balance
    bayer_pattern = 'RGGB'
    img_wb = WB_img(img_normalize, bayer_pattern, wb_r, wb_b)
    
    # Demosaic
    img_demosaic = torch.nn.functional.pixel_shuffle(img_wb.permute(2, 0, 1).unsqueeze(0), 2)
    img_demosaic = demosaic.forward(img_demosaic, pattern=bayer_pattern).squeeze(0).permute(1, 2, 0)
    img_demosaic = img_demosaic.clamp(0, 1)
    
    # Raw2xyz
    img_IXYZ = apply_cmatrix(img_demosaic, cam2xyz_realblur)

    # frome XYZ to linear RGB
    img_IL = xyz2lin(img_IXYZ)
    img_IL = torch.clamp(img_IL, 0, 1)
    
    return img_IL

In [ ]:
raw_path1 = 'notebooks/samples/CLN00001.ARW'
a7r3_raw1 = rawpy.imread(raw_path1).raw_image_visible.copy()[8:-8, 8:-8]
a7r3_lin1 = raw2Lin(a7r3_raw1,2576/1024.0, 1652/1024.0)

srgb_path1 = 'notebooks/samples/CLN00001.JPG'
a7r3_srgb1 = cv2.cvtColor(cv2.imread(srgb_path1, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB).astype('float32')/255

viz_two_images(a7r3_srgb1, a7r3_lin1, 'srgb1', 'lin1')

In [ ]:
raw_path2 = 'notebooks/samples/CLN00007.ARW'
a7r3_raw2 = rawpy.imread(raw_path2).raw_image_visible.copy()[8:-8, 8:-8]
a7r3_lin2 = raw2Lin(a7r3_raw2,2576/1024.0, 1652/1024.0)

srgb_path2 = 'notebooks/samples/CLN00007.JPG'
a7r3_srgb2 = cv2.cvtColor(cv2.imread(srgb_path2, cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB).astype('float32')/255

viz_two_images(a7r3_srgb2, a7r3_lin2, 'srgb2', 'lin2')

In [ ]:
colour_checker_data_srgb1 = detect_colour_checkers_segmentation(a7r3_srgb1, additional_data=True)
colour_checker_data_linrgb1 = detect_colour_checkers_segmentation(a7r3_lin1.numpy(), additional_data=True)

y1 = colour_checker_data_srgb1[0].swatch_colours
x1 = colour_checker_data_linrgb1[0].swatch_colours

colour_checker_data_srgb2 = detect_colour_checkers_segmentation(a7r3_srgb2, additional_data=True)
colour_checker_data_linrgb2 = detect_colour_checkers_segmentation(a7r3_lin2.numpy(), additional_data=True)

y2 = colour_checker_data_srgb2[0].swatch_colours
x2 = colour_checker_data_linrgb2[0].swatch_colours

x = np.concatenate([x1, x2], axis=0)
y = np.concatenate([y1, y2], axis=0)

In [ ]:
plt.scatter(x[:,0],y[:,0])
plt.scatter(x[:,1],y[:,1])
plt.scatter(x[:,2],y[:,2])

In [ ]:
# degree : 거듭제곱의 차수, include_bias : 편향값(1) 추가 여부
poly = PolynomialFeatures(degree=8, include_bias=False)

# 훈련 데이터셋 X_train 의 거듭제곱을 생성한 뒤, 훈련 데이터셋 X_train 에 새로운 변수로 추가
X_train_poly = poly.fit_transform(x.flatten()[:, None])
lin_reg = LinearRegression(fit_intercept=False)
lin_reg.fit(X_train_poly, y.flatten())

x_plot = np.linspace(0, 1, 200)
x_poly = poly.fit_transform(x_plot[:,None])
y_pred = lin_reg.predict(x_poly)
plt.plot(x_plot, y_pred)
plt.scatter(x.flatten(),y.flatten(),color='r',s=6)

#pred_our = lin2rgb_a7r3_polynomial(torch.tensor(x_plot))
#plt.plot(x_plot, pred_our)


<img src='./samples/Pipeline.png'>
이제 ISP에 필요한 모든 파라미터, CCM, CRF 를 추정했습니다. 최종 파이프라인은 다음과 같습니다.

# Camera ISP

In [ ]:

wb_r = 2576/1024.0 # R 에 대한 값
wb_g = 1.0
wb_b = 1652/1024.0 # B 에 대한 값

# White balance
bayer_pattern = 'RGGB'
img_wb = WB_img(img_normalize, bayer_pattern, wb_r, wb_b)

# Demosaic
img_demosaic = torch.nn.functional.pixel_shuffle(img_wb.permute(2, 0, 1).unsqueeze(0), 2)
img_demosaic = demosaic.forward(img_demosaic, pattern=bayer_pattern).squeeze(0).permute(1, 2, 0)
img_demosaic = img_demosaic.clamp(0, 1)

# Color conversion
img_IXYZ = img_demosaic @ cam2xyz_realblur_right

# from XYZ to linear RGB
img_IL = img_IXYZ @ M_xyz2lin
img_IL = torch.clamp(img_IL, 0, 1)

# linear RGB to sRGB
img_sRGB = lin2rgb_a7r3(img_IL)

viz_two_images(a7r3_srgb, img_sRGB, 'srgb', 'Our ISP pipeline')


# Conversion to RAW (Inverse of ISP) 

Inverse of ISP는 Camera ISP의 역과정


In [ ]:
# -------- INVERSE ISP PROCESS -------------------
# sRGB to linear
img_L = rgb2lin_a7r3(img_sRGB)

# Linear to XYZ
img_XYZ = img_L @ M_lin2xyz

# Color conversion XYZ to Raw
img_Cam = img_XYZ @ xyz2cam_realblur_right

# Mosaic
bayer_pattern = 'RGGB'
img_mosaic = mosaic_bayer(img_Cam, bayer_pattern)

wb_r = 2576/1024.0 # R 에 대한 값
wb_g = 1.0
wb_b = 1652/1024.0 # B 에 대한 값

#Inverse White balance
img_wb_inv = WB_img(img_mosaic, bayer_pattern, 1 / wb_r, 1 / wb_b)
img_wb_inv_viz = torch.nn.functional.pixel_shuffle(img_wb_inv.permute(2, 0, 1).unsqueeze(0), 2).squeeze(0).squeeze(0)
viz_two_images(a7r3_srgb[1570:1670, 2070:2170,:], img_wb_inv_viz[1570:1670, 2070:2170], 'srgb image', 'invese raw image')

In [ ]:
img_wb_inv.shape